In [ ]:
import argparse
import scanpy as sc
import numpy as np
import anndata

import time
import os

from sklearn.cluster import KMeans

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.sparse import csr_matrix, issparse

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import chi2
from matplotlib.patches import Ellipse

import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata

In [ ]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata.copy()
print('Normalizing...')
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e5)
sc.pp.log1p(adata_pp)
print('PCA...')
sc.pp.pca(adata_pp, n_comps=15)
print('Neighbors...')
sc.pp.neighbors(adata_pp)
print('Clustering...')
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.8)

In [ ]:
def merge_small_clusters(adata_pp, min_cluster_size):
    
    while not (all(clust_size >= min_cluster_size for clust_size in adata_pp.obs.groups.value_counts() if clust_size != 0)):
    
        c_counts = adata_pp.obs.groups.value_counts()
        cluster_list = list(c_counts.keys())

        merge_dict = {}

        for c in cluster_list:

            if c_counts[c] < min_cluster_size:

                #find closets cluster
                avg_distances = {}
                k_clusts = [l for l in cluster_list if l!=c]
                for k in k_clusts:
                    c_cells = adata_pp[adata_pp.obs['groups']==c,:].obsm['X_pca']
                    k_cells = adata_pp[adata_pp.obs['groups']==k,:].obsm['X_pca']
                    dist_mat = distance.cdist(c_cells, k_cells, metric='cosine')
                    mean_val = np.mean(dist_mat[np.triu_indices(n=len(c_cells), m=len(k_cells) , k = 1)])
                    avg_distances[k] = mean_val

                min_k = min(avg_distances, key=avg_distances.get)
                merge_dict[c]=min_k

        for c in list(merge_dict.keys()):
            adata_pp.obs.loc[adata_pp.obs['groups']==c,'groups'] = merge_dict[c]
        
    # reset categories
    adata_pp.obs['groups'] = adata_pp.obs['groups'].astype('str').astype('category')
    
    return adata_pp

In [ ]:
merge_small_clusters(adata_pp, 100)

In [ ]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
adata.obs['norm_groups'] = input_groups
adata.obs['norm_groups'].value_counts()

In [ ]:
adata

In [ ]:
import scranPY
scranPY.compute_sum_factors(adata, clusters='norm_groups', normalize_counts=True)